<a href="https://colab.research.google.com/github/hereagain-Y/test-only/blob/main/Extact_deccode_layere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

from six.moves import xrange
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cuda = True
DEVICE = torch.device("cuda" if cuda else "cpu")
batch_size = 1000

x_dim=114 # 19*6
hidden_dim = 100
#hidden_dim2 = 

latent_dim = 64

lr = 1e-3

epochs = 500

In [6]:

class Encoder(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()

        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        #self.FC_input3 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_mean  = nn.Linear(hidden_dim, latent_dim)
        self.FC_var   = nn.Linear (hidden_dim, latent_dim)
        
       # self.LeakyReLU = nn.LeakyReLU(0.2)
        self.relu = nn.ReLU()
        
        self.training = True
        
    def forward(self, x):
        h_       = self.relu(self.FC_input(x))
        h_       = self.relu(self.FC_input2(h_))
        #h_       = self.relu(self.FC_input3(h_))
        mean     = self.FC_mean(h_)
        log_var  = self.FC_var(h_)                     # encoder produces mean and log of variance 
                                                     #             (i.e., parateters of simple tractable normal distribution "q"
        
        return mean, log_var

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim,output_dim):
        super(Decoder, self).__init__()
        self.FC_hidden = nn.Linear(latent_dim, hidden_dim)
        #self.FC_hidden3 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, output_dim)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        h     = self.relu(self.FC_hidden(x))
        #h     = self.relu(self.FC_hidden3(h))
        h     = self.relu(self.FC_hidden2(h))
        #x_hat = torch.sigmoid(self.FC_output(h))
        x_hat = self.relu(self.FC_output(h))
        
        #x_hat = torch.sigmoid(self.FC_output(h))
        return x_hat

class Model(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Model, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
        
    def reparameterization(self, mean, var):
        #epsilon = torch.randn_like(var).to(DEVICE)        # sampling epsilon        
        #z = mean + var*epsilon 
        z  = mean                       # reparameterization trick
        return z
        
                
    def forward(self, x):
        mean, log_var = self.Encoder(x)
        z = self.reparameterization(mean, torch.exp(0.5 * log_var)) # takes exponential function (log var -> var) ]
        x_hat            = self.Decoder(z)
        
        return x_hat, mean, log_var

encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim,  latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim = x_dim)

model = Model(Encoder=encoder, Decoder=decoder).to(DEVICE)

In [7]:
model = torch.load('/content/drive/My Drive/DL/VAE/Newdata/NewPCA_VAE_modify_MSE_model_10000_echo.apx')

In [8]:
seq_test = pd.read_csv("/content/drive/My Drive/DL/VAE/NormalCDR3_test.txt", delimiter='\t',header=None,names=['seq'])

In [9]:
seq_test['length'] = [len(seq) for seq in seq_test['seq']]

seq_test = seq_test[ seq_test['length']<=20 ]
seq = list( seq_test['seq'] )


In [10]:
d= {} # bipphysics dictionary
with open("/content/drive/My Drive/DL/VAE/Newdata/PC1-6_index.txt") as f:
    next(f)
    for line in f.readlines():
        line=line.strip().split('\t') #
        AA=line[0]
        tag=0
        values=[]
        for PC in line[1:]:
            values.append(float(PC))
        if tag==1: 
            continue
        d[AA]=values


In [11]:
AAs= ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
d=dict(zip(AAs,list(d.values())))

In [12]:

def AAindexEncoding(Seq):
    length_seq=len(Seq)
    global max_len
    AAE=np.zeros([19,6])
    if length_seq<19:
        for amino in range(length_seq):
            AA=Seq[amino]# 
            AAE[amino,]=d[AA] # add PC value 
            
        for amino in range(length_seq,19):
            AAE[amino,]=np.zeros(6)
    else: 
        for amino in range(length_seq): # zero padding
            AA=Seq[amino]# 
            AAE[amino,]=d[AA]
        
    #AAE=np.transpose(AAE.astype(np.float32)) # row as PC. and column as AA sequence 
    return AAE 
  
def GetFeatures(file):
    #sequence=file['amino_acid'].tolist()
    #sequence=np.array(sequence)
    #sequence = file.read().splitlines()
    #sequence=np.array(sequence)
    hot_encode=[]
    for seq in file:
        hot_encode.append(AAindexEncoding(seq))
    hot_encode=np.array(hot_encode)
    result=np.array(hot_encode)
    return(result) # dimension: number of sequence [15*length(sequence)]

r1=GetFeatures(seq)

get encode layer 

In [14]:
r1_transform=torch.from_numpy(r1).float() # change to tensor and float 
m2=r1_transform.view(len(r1_transform),114) # flatten 
#m2 = torch.from_numpy(m2).float()
#result = model(m2)[0] # m
#result = model(m2.cuda())[0]#
#result2 = result.view( len(result),19,6 ).cpu().detach().numpy()

In [19]:
model(m2.cuda())[0].size() #19851 *114

torch.Size([19851, 114])

In [20]:
model(m2.cuda())[1].size()  # 64 latent

torch.Size([19851, 64])

In [44]:
model(m2.cuda())[2].size() # 19851*64

torch.Size([19851, 64])

In [25]:
inter= model.Encoder(m2.cuda()) # use the encode layer

In [36]:
m1=inter[0]# latent mean layer

In [38]:
model.Decoder(m1).size() # decode to orginal dimension 

torch.Size([19851, 114])

In [40]:
model(m2.cuda())[1] #. exacltly same as m1 model.Encoder(m2.cuda()) [0]

tensor([[ 4.7977e-05,  6.8177e-06, -4.0675e-05,  ..., -3.6227e-04,
         -5.8922e-05,  3.6709e-05],
        [ 4.8089e-05,  6.7758e-06, -4.0291e-05,  ..., -3.4761e-04,
         -5.8473e-05,  3.6405e-05],
        [ 4.7965e-05,  6.5775e-06, -4.0668e-05,  ..., -3.3977e-04,
         -5.8597e-05,  3.6286e-05],
        ...,
        [ 5.1204e-05,  8.8991e-06, -4.1294e-05,  ...,  6.0574e-04,
         -5.6253e-05,  1.3266e-05],
        [ 5.0246e-05,  8.1521e-06, -4.1348e-05,  ...,  3.5192e-04,
         -5.6972e-05,  1.9486e-05],
        [ 5.0212e-05,  8.3799e-06, -4.1795e-05,  ...,  3.4880e-04,
         -5.7575e-05,  1.9440e-05]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [42]:
model(m2.cuda())[2]

tensor([[-2.4608e-05,  1.9589e-05,  5.9137e-08,  ..., -3.9671e-08,
         -3.1551e-06,  1.1839e-08],
        [-2.4544e-05,  1.9856e-05,  5.9608e-08,  ..., -3.9932e-08,
         -3.1597e-06,  9.4138e-09],
        [-2.4716e-05,  1.9979e-05,  5.9292e-08,  ..., -3.9812e-08,
         -3.1703e-06,  1.1567e-08],
        ...,
        [-2.4881e-05,  2.0466e-05,  7.4047e-08,  ..., -5.2987e-08,
         -3.3125e-06,  1.7405e-08],
        [-2.4936e-05,  2.0399e-05,  6.9830e-08,  ..., -4.9317e-08,
         -3.2797e-06,  1.7100e-08],
        [-2.4957e-05,  1.9918e-05,  6.9611e-08,  ..., -4.9298e-08,
         -3.2707e-06,  2.0084e-08]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [45]:
model.Decoder(m1).size() # 114 

torch.Size([19851, 114])